In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/aiedu-courses/all_datasets/refs/heads/main/Womens%20Clothing%20E-Commerce%20Reviews.csv')

In [ ]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
dataset = df[['Review Text', 'Recommended IND']]
dataset.head()

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [ ]:
dataset['Recommended IND'].value_counts()

,count
Recommended IND,
1,19314
0,4172


In [ ]:
#Для задач бинарной классификации стоп-слова часто не удаляют, потому что:
#Контекст: В коротких отзывах (таких как в твоем примере) даже стоп-слова могут иметь значение в контексте.
#Новые модели (например, трансформеры) обычно обучаются на необработанных текстах, где стоп-слова присутствуют, и они могут незначительно помогать в процессе обучения.
#Поэтому мы напишем простую функцию для предобработки без использования nltk

In [ ]:
import re

def clear_text(text):
  text = str(text)
  text = text.lower()

  # Удаление эмодзи
  text = text.encode('ascii', 'ignore').decode('ascii')

  # Удаление всех символов, кроме букв и пробелов
  text = re.sub(r'[^A-Za-z\s]', '', text)

  # Удаление лишних пробелов
  text = re.sub(r"\s+", " ", text).strip()

  return text



In [ ]:
dataset.loc[:, 'Review Text'] = dataset['Review Text'].apply(clear_text)
dataset.head()

,Review Text,Recommended IND
0,absolutely wonderful silky and sexy and comfor...,1
1,love this dress its sooo pretty i happened to ...,1
2,i had such high hopes for this dress and reall...,0
3,i love love love this jumpsuit its fun flirty ...,1
4,this shirt is very flattering to all due to th...,1


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)

# изначально эта модель создана для других задач, но если обернуть ее в класс AutoModelForSequenceClassification,
#то модель подставляет стандартную голову классификации, но она не обучена для этой задачи.
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

#Эта модель НЕ ОБУЧЕНА на классификацию – она просто использует случайно инициализированную классификационную голову.

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
tokenized = dataset['Review Text'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True))
tokenized.head()

,Review Text
0,"[input_ids, attention_mask]"
1,"[input_ids, attention_mask]"
2,"[input_ids, attention_mask]"
3,"[input_ids, attention_mask]"
4,"[input_ids, attention_mask]"


In [ ]:
# Извлекаем input_ids и attention_mask
dataset.loc[:, 'input_ids'] = tokenized.apply(lambda x: x['input_ids'])
dataset.loc[:, 'attention_mask'] = tokenized.apply(lambda x: x['attention_mask'])

<ipython-input-12-d4ea95bd6a65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'input_ids'] = tokenized.apply(lambda x: x['input_ids'])
<ipython-input-12-d4ea95bd6a65>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'attention_mask'] = tokenized.apply(lambda x: x['attention_mask'])


In [ ]:
dataset = dataset.drop(columns='Review Text')
dataset.rename(columns={'Recommended IND': 'labels'}, inplace=True)
dataset.head()

,labels,input_ids,attention_mask
0,1,"[101, 7078, 6919, 18848, 1998, 7916, 1998, 662...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
1,1,"[101, 2293, 2023, 4377, 2049, 17111, 2080, 349...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,0,"[101, 1045, 2018, 2107, 2152, 8069, 2005, 2023...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,"[101, 1045, 2293, 2293, 2293, 2023, 14523, 146...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[101, 2023, 3797, 2003, 2200, 4257, 17989, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
#!pip install datasets

In [ ]:
#Модели Hugging Face и их инструменты (например, Trainer) напрямую работают с объектами типа Dataset
from datasets import Dataset

ds = Dataset.from_pandas(dataset)
ds

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 23486
})

In [ ]:
print(ds.features)

{'labels': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [ ]:
ds = ds.train_test_split(test_size=0.2)
ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 18788
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4698
    })
})

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',              # Директория для сохранения модели и логов
    num_train_epochs=3,                  # Количество эпох
    per_device_train_batch_size=8,       # Размер батча для тренировки
    per_device_eval_batch_size=16,       # Размер батча для валидации
    warmup_steps=500,                    # Количество шагов для разогрева
    weight_decay=0.01,                   # Вес для регуляризации
    #logging_dir='./logs',                # Директория для логов
    #logging_steps=10,                    # Частота логирования
    eval_strategy="epoch",               # Стратегия для валидации: на каждую эпоху
    save_strategy="epoch",               # Сохранение модели: на каждую эпоху
    load_best_model_at_end=True,         # Загрузка лучшей модели после обучения
    metric_for_best_model="f1",          # Метрика для выбора лучшей модели
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from sklearn.metrics import f1_score
import numpy as np


# передается в Trainer с помощью аргумента compute_metrics, Trainer будет вызывать её после каждого шага или эпохи.
#Trainer автоматически подаст в эту функцию два аргумента: predictions и labels.

def compute_metrics(p):
    predictions, labels = p #[2.5, 0.3, -1.2] - логит
    preds = np.argmax(predictions, axis=1)  # Преобразуем логиты в предсказания

    preds = preds.cpu().numpy() if torch.is_tensor(preds) else preds
    labels = labels.cpu().numpy() if torch.is_tensor(labels) else labels

    f1 = f1_score(labels, preds, average='weighted')  # Вычисляем F1-score
    return {"f1": f1}

In [ ]:
trainer = Trainer(
    model=model,                  # модель
    args=training_args,           # Аргументы обучения
    train_dataset=ds["train"],    # Обучающая выборка
    eval_dataset=ds["test"],      # Тестовая выборка
    compute_metrics=compute_metrics  # Функция вычисления метрик
)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.272000,0.406276,0.886592
2,0.198600,0.244186,0.915514
3,0.122100,0.348642,0.915906


TrainOutput(global_step=7047, training_loss=0.21278475070821828, metrics={'train_runtime': 2845.1066, 'train_samples_per_second': 19.811, 'train_steps_per_second': 2.477, 'total_flos': 7466392457846784.0, 'train_loss': 0.21278475070821828, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Затем сохранить модель
os.makedirs('/content/drive/MyDrive/my_models/2025-04-02', exist_ok=True)
trainer.save_model('/content/drive/MyDrive/my_models/2025-04-02/my_model')

In [ ]:
inputs = tokenizer(
    "This is outrageous, i have never expected anything like this",
    return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  2023,  2003, 25506,  1010,  1045,  2031,  2196,  3517,  2505,
          2066,  2023,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# Переносим входные данные на GPU
inputs = {k: v.to('cuda') for k, v in inputs.items()}

In [ ]:
#поскольку я забыл добавить softmax в классификационную голову, то модель будет возвращать логиты
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.6590, -2.1699]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
#Модель возвращает логиты вместо вероятностей, потому что:
#Логиты удобнее для обучения (они стабильнее численно)
#Софтмакс обычно применяется во время обучения внутри loss function
#Это дает больше гибкости для разных типов задач

probabilities = torch.softmax(outputs.logits, dim=-1)
probabilities

tensor([[0.9921, 0.0079]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [ ]:
# Получаем индекс класса с максимальной вероятностью
predicted_class = torch.argmax(probabilities, dim=-1)
predicted_class

tensor([0], device='cuda:0')

In [ ]:
#УРАААААА